# [Advent of Code 2019](https://adventofcode.com/2019)

# The toolbox

Generalised pieces of code that either can be used in multiple questions or that simply makes understand the implementation easier.

In [1]:
import math


def Input(day):
    """Fetch the data input from disk."""
    filename = f'../data/advent2019/input{day}.txt'
    return open(filename)


def run_intcode(memory):
    """Run intcode operations.
    
    Will return the final state of the memory once all the operations have
    been run.
    """
    # Clone the memory list so we can safely mutate
    memory = memory[:]

    instruction_pointer = 0
    
    # Available operations, each returning the updated instruction pointer
    def add():
        instruction_length = 4
        _, *params = memory[instruction_pointer:instruction_pointer + instruction_length]
        a, b, c = params
        memory[c] = memory[a] + memory[b]        
        return instruction_pointer + instruction_length
    
    def multiply():
        instruction_length = 4
        _, *params = memory[instruction_pointer:instruction_pointer + instruction_length]
        a, b, c = params
        memory[c] = memory[a] * memory[b]        
        return instruction_pointer + instruction_length
    
    def stop():
        raise StopIteration()
    
    # Assign each operation to its given instruction number
    operations = {
        1: add,
        2: multiply,
        99: stop,
    }
    
    try:
        while True:
            instruction = memory[instruction_pointer]
            operation = operations[instruction]
            instruction_pointer = operation()
    except StopIteration:
        pass

    return memory

# [Day 1](https://adventofcode.com/2019/day/1)

Things start off pretty simply, just summing a list after applying some rounding.

In [2]:
def get_modules():
    """Utility for loading the modules as integers."""
    with Input(1) as f:
        modules = f.readlines()
    return [int(module) for module in modules]
        

def fuel_required(mass):
    fuel = math.floor(mass / 3) - 2
    return max(0, fuel)


assert fuel_required(12) == 2
assert fuel_required(100756) == 33583
assert fuel_required(1) == 0

In [3]:
modules = get_modules()
sum(fuel_required(module) for module in modules)

3401852

Now we have to account for the additional mass we're adding to the ship as fuel. 

In [4]:
def fuel_required_including_fuel(mass):
    total = fuel_required(mass)
    additional_mass = total
    
    while additional_mass:
        additional_mass = fuel_required(additional_mass)
        total += additional_mass
    
    return total
    

assert fuel_required_including_fuel(1969) == 966

In [5]:
sum(fuel_required_including_fuel(module) for module in modules)

5099916

# [Day 2](https://adventofcode.com/2019/day/2)

First time we need to use some code that the question sepecifically tells us we'll need later (hence it going in the toolbox above).

In [6]:
assert run_intcode([2,4,4,5,99,0]) == [2,4,4,5,99,9801]
assert run_intcode([1,1,1,4,99,5,6,0,99]) == [30,1,1,4,2,5,6,0,99]

In [7]:
def parse_input():
    with Input(2) as f:
        data = f.read()
    return [int(d) for d in data.split(',')]


op_codes = parse_input()

# replace the codes the question tells you to
op_codes[1] = 12
op_codes[2] = 2

run_intcode(op_codes)[0]

5305097

In [8]:
def cycle_nouns_and_verbs(op_codes):
    term_value = 19690720
    for noun in range(100):
        for verb in range(100):
            op_codes[1] = noun
            op_codes[2] = verb

            output = run_intcode(op_codes)[0]

            if output == term_value:
                return 100 * noun + verb
    else:
        raise Exception('No solution found.')


cycle_nouns_and_verbs(op_codes)

4925